In [ ]:
!pip install --ignore-installed Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.7.0 requires Pillow<9.6,>=9.3, but you have pillow 9.0.0 which is incompatible.


In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.0.0"
!pip install --pre autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/df_drama.xlsx')

In [ ]:
df = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
df.head()

,year,Drama,type_Movie,type_show,cast_no_cast,Brazil,Canada,Egypt,France,India,...,Philippines,duration_season_1,duration_seasons_2,duration_seasons_3,duration_seasons_4,rating_pg_13,rating_r,rating_tv_14,rating_tv_ma,rating_tv_pg
0,2021,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2021,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2021,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,2021,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,2021,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [ ]:

#Оставляем в датасете 95% данных для дальнейшего моделирования
df_pc = df.sample(frac=0.95,random_state=1)
#Выделяем 5% данных для предсказаний
df_pc_us = df.drop(df_pc.index)
#Обновляем индексы
df_pc.reset_index(inplace=True, drop=True)
df_pc_us.reset_index(inplace=True, drop=True)
#Проверяем разбиение
print('Исходные данные: ' + str(df.shape))
print('Данные для моделирования: ' + str(df_pc.shape))
print('Данные для предсказаний: ' + str(df_pc_us.shape))
df_pc.head(2)

Исходные данные: (8790, 31)
Данные для моделирования: (8350, 31)
Данные для предсказаний: (440, 31)


,year,Drama,type_Movie,type_show,cast_no_cast,Brazil,Canada,Egypt,France,India,...,Philippines,duration_season_1,duration_seasons_2,duration_seasons_3,duration_seasons_4,rating_pg_13,rating_r,rating_tv_14,rating_tv_ma,rating_tv_pg
0,2018,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2016,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
label = 'Drama'
print("Summary of class variable: \n", df_pc[label].describe())

Summary of class variable: 
 count    8350.000000
mean        0.182036
std         0.385897
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Drama, dtype: float64


In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(df_pc)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.7.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    8350
Train Data Columns: 30
Label Column: Drama
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12044.64 MB
	Train Data (Original)  Memory Usage: 2.0 MB (0.0% of available memory)
	Inferring data type of each feature based on c

In [ ]:
y_test = df_pc_us['Drama']  # values to predict
# delete label column to prove we're not cheating
test_data_nolab = df_pc_us.drop(columns=['Drama'])  
test_data_nolab.head()

,year,type_Movie,type_show,cast_no_cast,Brazil,Canada,Egypt,France,India,Indonesia,...,Philippines,duration_season_1,duration_seasons_2,duration_seasons_3,duration_seasons_4,rating_pg_13,rating_r,rating_tv_14,rating_tv_ma,rating_tv_pg
0,2021,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,2021,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,2021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#predictor = TabularPredictor.load(save_path) 
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Evaluation: accuracy on test data: 0.8045454545454546
Evaluations on test data:
{
    "accuracy": 0.8045454545454546,
    "balanced_accuracy": 0.5100809986324906,
    "mcc": 0.04264426206004,
    "f1": 0.0851063829787234,
    "precision": 0.26666666666666666,
    "recall": 0.05063291139240506
}


Predictions:  
 0      0
1      0
2      0
3      0
4      0
      ..
435    0
436    0
437    0
438    0
439    0
Name: Drama, Length: 440, dtype: int64


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
predictor.leaderboard(df_pc_us)

                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              LightGBM    0.815909   0.823952        0.004003       0.027218   2.737928                 0.004003                0.027218           2.737928            1       True          4
1              CatBoost    0.815909   0.823952        0.005965       0.005921   2.928114                 0.005965                0.005921           2.928114            1       True          7
2        NeuralNetTorch    0.813636   0.827545        0.010724       0.010999  17.017961                 0.010724                0.010999          17.017961            1       True         12
3         LightGBMLarge    0.813636   0.823952        0.016158       0.010770   1.100065                 0.016158                0.010770           1.100065            1       True         13
4       NeuralNetFastAI    0.809091   0.

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.815909,0.823952,0.004003,0.027218,2.737928,0.004003,0.027218,2.737928,1,True,4
1,CatBoost,0.815909,0.823952,0.005965,0.005921,2.928114,0.005965,0.005921,2.928114,1,True,7
2,NeuralNetTorch,0.813636,0.827545,0.010724,0.010999,17.017961,0.010724,0.010999,17.017961,1,True,12
3,LightGBMLarge,0.813636,0.823952,0.016158,0.010770,1.100065,0.016158,0.010770,1.100065,1,True,13
4,NeuralNetFastAI,0.809091,0.826347,0.028946,0.023864,25.244392,0.028946,0.023864,25.244392,1,True,10
5,RandomForestEntr,0.809091,0.816766,0.119467,0.287055,4.561681,0.119467,0.287055,4.561681,1,True,6
6,RandomForestGini,0.809091,0.816766,0.126619,0.228498,4.111594,0.126619,0.228498,4.111594,1,True,5
7,ExtraTreesGini,0.806818,0.815569,0.124969,0.156039,2.641977,0.124969,0.156039,2.641977,1,True,8
8,ExtraTreesEntr,0.806818,0.815569,0.127242,0.182818,2.555551,0.127242,0.182818,2.555551,1,True,9
9,LightGBMXT,0.804545,0.825150,0.021438,0.090242,1.809615,0.021438,0.090242,1.809615,1,True,3


In [ ]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.828743       0.241687  21.876352                0.002189           0.746796            2       True         14
1        NeuralNetTorch   0.827545       0.010999  17.017961                0.010999          17.017961            1       True         12
2               XGBoost   0.826347       0.015315   1.385137                0.015315           1.385137            1       True         11
3       NeuralNetFastAI   0.826347       0.023864  25.244392                0.023864          25.244392            1       True         10
4            LightGBMXT   0.825150       0.090242   1.809615                0.090242           1.809615            1       True          3
5              CatBoost   0.823952       0.005921   2.928114                0.005921           2.928114 

{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.7077844311377246,
  'KNeighborsDist': 0.7089820359281437,
  'LightGBMXT': 0.8251497005988024,
  'LightGBM': 0.8239520958083832,
  'RandomForestGini': 0.8167664670658683,
  'RandomForestEntr': 0.8167664670658683,
  'CatBoost': 0.8239520958083832,
  'ExtraTreesGini': 0.8155688622754491,
  'ExtraTreesEntr': 0.8155688622754491,
  'NeuralNetFastAI': 0.8263473053892215,
  'XGBoost': 0.8263473053892215,
  'NeuralNetTorch': 0.8275449101796407,
  'LightGBMLarge': 

In [ ]:
predictor.feature_importance(df_pc_us)

Computing feature importance via permutation shuffling for 30 features using 440 rows with 5 shuffle sets...
	25.51s	= Expected runtime (5.1s per shuffle set)
	2.94s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
rating_tv_pg,4.545455e-03,0.001607,0.001599,5,0.007854,0.001236
Turkey,4.545455e-04,0.001016,0.186950,5,0.002547,-0.001638
duration_seasons_2,4.545455e-04,0.001016,0.186950,5,0.002547,-0.001638
France,2.220446e-17,0.001607,0.500000,5,0.003309,-0.003309
duration_seasons_4,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
Brazil,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
Egypt,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
Mexico,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
Japan,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
cast_no_cast,-4.545455e-04,0.005883,0.564390,5,0.011658,-0.012567


In [ ]:
i = 4  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred = predictor.predict(df_pc_us, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

Prediction from RandomForestGini model: 0


In [ ]:
all_models = predictor.get_model_names()
model_to_use = all_models [i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

In [ ]:
predictor_information

{'path': 'agModels-predictClass/',
 'label': 'Drama',
 'random_state': 0,
 'version': '0.7.0',
 'features': ['year',
  'type_Movie',
  'type_show',
  'cast_no_cast',
  'Brazil',
  'Canada',
  'Egypt',
  'France',
  'India',
  'Indonesia',
  'Japan',
  'Nigeria',
  'South_Korea',
  'Spain',
  'USA',
  'UK',
  'Mexico',
  'Turkey',
  'Australia',
  'Taiwan',
  'Philippines',
  'duration_season_1',
  'duration_seasons_2',
  'duration_seasons_3',
  'duration_seasons_4',
  'rating_pg_13',
  'rating_r',
  'rating_tv_14',
  'rating_tv_ma',
  'rating_tv_pg'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x7fea80695640>,
 'time_fit_preprocessing': 0.3712592124938965,
 'time_fit_training': 69.19347977638245,
 'time_fit_total': 69.56473898887634,
 'time_limit': None,
 'time_train_start': 1677413521.9262335,
 'num_rows_train': 7515,
 'num_cols_train': 30,
 'num_rows_val': 835,
 'num_classes': 2,
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'best

In [ ]:
import numpy as np

In [ ]:
data=pd.DataFrame()
preds = np.array(y_pred)
data['predict']=y_pred

In [ ]:
data

,predict
0,0
1,0
2,0
3,0
4,0
...,...
435,0
436,0
437,0
438,0


In [ ]:
wine_predict = df_pc_us.join(data)

In [ ]:
wine_predict

,year,Drama,type_Movie,type_show,cast_no_cast,Brazil,Canada,Egypt,France,India,...,duration_season_1,duration_seasons_2,duration_seasons_3,duration_seasons_4,rating_pg_13,rating_r,rating_tv_14,rating_tv_ma,rating_tv_pg,predict
0,2021,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,2021,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,2021,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2021,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,2018,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
436,2018,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
437,2018,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
438,2018,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
agcm=confusion_matrix(y_test, y_pred,normalize='true')
dispag = ConfusionMatrixDisplay(agcm, display_labels=['0','1'])
dispag.plot()

AttributeError: ignored

<Figure size 432x288 with 2 Axes>

In [ ]:
agcm

In [ ]:
import matplotlib.pyplot as plt